In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

In [2]:
import numpy as np
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. 先把資料全部讀進來 (記得適當調整 memory footprint)
def load_data(generator, num_samples):
    X, y = [], []
    for batch_x, batch_y in generator:
        X.append(batch_x); y.append(batch_y)
        if len(X)*generator.batch_size >= num_samples:
            break
    X = np.vstack(X); y = np.concatenate(y)
    return X, y

# 2. 包裝 CNN 結構成函式
def build_model(input_shape=(128,128,3)):
    m = models.Sequential([
        layers.Conv2D(64, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    m.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return m

# 3. 建立 ImageDataGenerator 與 generator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=15,
                                   width_shift_range=0.1,
                                   zoom_range=0.1,
                                   fill_mode='nearest')
test_datagen  = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    'train', target_size=(128,128), batch_size=32, class_mode='binary'
)
test_gen = test_datagen.flow_from_directory(
    'test',  target_size=(128,128), batch_size=32, class_mode='binary', shuffle=False
)

# 4. 把資料一次 load 出來
X_train, y_train = load_data(train_gen, train_gen.samples)
X_test,  y_test  = load_data(test_gen,  test_gen.samples)

# 5. Bagging 參數
n_estimators = 5
ensemble = []

# 6. 針對每個子模型做 bootstrap 取樣並訓練
for i in range(n_estimators):
    print(f"Training estimator #{i+1}")
    # Bootstrap sample
    idx = np.random.choice(len(X_train), size=len(X_train), replace=True)
    X_bs, y_bs = X_train[idx], y_train[idx]
    
    # 建立、訓練子模型
    m = build_model(input_shape=(128,128,3))
    m.fit(
        X_bs, y_bs,
        epochs=10,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=1
    )
    ensemble.append(m)

# 7. 預測並平均機率
all_preds = np.stack([m.predict(X_test).ravel() for m in ensemble], axis=0)
avg_prob  = np.mean(all_preds, axis=0)
y_pred    = (avg_prob > 0.5).astype(int)

# 8. 計算整體準確率
accuracy = np.mean(y_pred == y_test)
print(f"Bagging ensemble test accuracy: {accuracy:.4f}")


Found 220 images belonging to 2 classes.
Found 80 images belonging to 2 classes.
Training estimator #1
Epoch 1/10
7/7 [==============================] - 3s 72ms/step - loss: 0.8867 - accuracy: 0.4364 - val_loss: 0.6935 - val_accuracy: 0.5000
Epoch 2/10
7/7 [==============================] - 0s 21ms/step - loss: 0.6752 - accuracy: 0.5500 - val_loss: 0.7788 - val_accuracy: 0.5000
Epoch 3/10
7/7 [==============================] - 0s 21ms/step - loss: 0.6484 - accuracy: 0.6136 - val_loss: 0.7260 - val_accuracy: 0.4625
Epoch 4/10
7/7 [==============================] - 0s 22ms/step - loss: 0.5408 - accuracy: 0.8136 - val_loss: 0.7527 - val_accuracy: 0.4750
Epoch 5/10
7/7 [==============================] - 0s 20ms/step - loss: 0.4027 - accuracy: 0.8727 - val_loss: 0.8854 - val_accuracy: 0.5375
Epoch 6/10
7/7 [==============================] - 0s 21ms/step - loss: 0.2852 - accuracy: 0.9000 - val_loss: 1.0884 - val_accuracy: 0.5500
Epoch 7/10
7/7 [==============================] - 0s 20ms/step 